<a href="https://colab.research.google.com/github/NLP-END3/Session11/blob/main/BART_paraphrasing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simpletransformers

In [2]:
%%bash
mkdir data
wget https://storage.googleapis.com/paws/english/paws_wiki_labeled_final.tar.gz -P data
tar -xvf data/paws_wiki_labeled_final.tar.gz -C data
mv data/final/* data
rm -r data/final

wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv -P data

final/test.tsv
final/
final/train.tsv
final/dev.tsv


mkdir: cannot create directory ‘data’: File exists
--2022-01-28 08:10:06--  https://storage.googleapis.com/paws/english/paws_wiki_labeled_final.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.97.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4687157 (4.5M) [application/gzip]
Saving to: ‘data/paws_wiki_labeled_final.tar.gz.1’

     0K .......... .......... .......... .......... ..........  1% 69.2M 0s
    50K .......... .......... .......... .......... ..........  2% 57.9M 0s
   100K .......... .......... .......... .......... ..........  3% 88.3M 0s
   150K .......... .......... .......... .......... ..........  4% 79.4M 0s
   200K .......... .......... .......... .......... ..........  5% 73.9M 0s
   250K .......... .......... .......... .......... ..........  6%  101M 0s
   300K .......... .......... .......... .

#Utils.py

In [3]:
import warnings

import pandas as pd


def load_data(
    file_path, input_text_column, target_text_column, label_column, keep_label=1
):
    df = pd.read_csv(file_path, sep="\t", error_bad_lines=False)
    df = df.loc[df[label_column] == keep_label]
    df = df.rename(
        columns={input_text_column: "input_text", target_text_column: "target_text"}
    )
    df = df[["input_text", "target_text"]]
    df["prefix"] = "paraphrase"

    return df


def clean_unnecessary_spaces(out_string):
    if not isinstance(out_string, str):
        warnings.warn(f">>> {out_string} <<< is not a string.")
        out_string = str(out_string)
    out_string = (
        out_string.replace(" .", ".")
        .replace(" ?", "?")
        .replace(" !", "!")
        .replace(" ,", ",")
        .replace(" ' ", "'")
        .replace(" n't", "n't")
        .replace(" 'm", "'m")
        .replace(" 's", "'s")
        .replace(" 've", "'ve")
        .replace(" 're", "'re")
    )
    return out_string

In [4]:
import os
from datetime import datetime
import logging

import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

#from utils import load_data, clean_unnecessary_spaces


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

# Load Datasets

In [5]:
# Google Data
train_df = pd.read_csv("data/train.tsv", sep="\t").astype(str)
eval_df = pd.read_csv("data/dev.tsv", sep="\t").astype(str)

train_df = train_df.loc[train_df["label"] == "1"]
eval_df = eval_df.loc[eval_df["label"] == "1"]

train_df = train_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)
eval_df = eval_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)

train_df = train_df[["input_text", "target_text"]]
eval_df = eval_df[["input_text", "target_text"]]

train_df["prefix"] = "paraphrase"
eval_df["prefix"] = "paraphrase"

# # MSRP Data
# train_df = pd.concat(
#     [
#         train_df,
#         load_data("data/msr_paraphrase_train.txt", "#1 String", "#2 String", "Quality"),
#     ]
# )
# eval_df = pd.concat(
#     [
#         eval_df,
#         load_data("data/msr_paraphrase_test.txt", "#1 String", "#2 String", "Quality"),
#     ]
# )

# # Quora Data

# # The Quora Dataset is not separated into train/test, so we do it manually the first time.
# df = load_data(
#     "data/quora_duplicate_questions.tsv", "question1", "question2", "is_duplicate"
# )
# q_train, q_test = train_test_split(df)

# q_train.to_csv("data/quora_train.tsv", sep="\t")
# q_test.to_csv("data/quora_test.tsv", sep="\t")

# # The code block above only needs to be run once.
# # After that, the two lines below are sufficient to load the Quora dataset.

# # q_train = pd.read_csv("data/quora_train.tsv", sep="\t")
# # q_test = pd.read_csv("data/quora_test.tsv", sep="\t")

# train_df = pd.concat([train_df, q_train])
# eval_df = pd.concat([eval_df, q_test])

train_df = train_df[["prefix", "input_text", "target_text"]]
eval_df = eval_df[["prefix", "input_text", "target_text"]]

train_df = train_df.dropna()
eval_df = eval_df.dropna()

train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)

eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)

print(train_df)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


           prefix  ...                                        target_text
1      paraphrase  ...  The 1975 -- 76 season of the National Basketba...
3      paraphrase  ...  The results are high when comparable flow rate...
4      paraphrase  ...  It is the seat of the district of Zerendi in A...
5      paraphrase  ...  William Henry Harman was born in Waynesboro, V...
7      paraphrase  ...  Given a discrete set of probabilities formula ...
...           ...  ...                                                ...
49384  paraphrase  ...  The Romance language currently spoken in Galic...
49390  paraphrase  ...  It is necessary to note that k is a vector con...
49393  paraphrase  ...  Tim Tim Henman won 6 -- 2, 7 -- 6 against Yevg...
49395  paraphrase  ...  He was considered an active member of the Coun...
49397  paraphrase  ...  She was at Cork on 24 June, and arrived in the...

[21829 rows x 3 columns]


In [6]:
import torch
torch.cuda.empty_cache()

In [7]:
model_args = Seq2SeqArgs()
model_args.do_sample = True
model_args.eval_batch_size = 8 #64
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = True #False ########## Changed at 7pm 26-Jan-2022
model_args.learning_rate = 5e-5
model_args.max_length = 128
model_args.max_seq_length = 128
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.num_train_epochs = 2
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.top_k = 50
model_args.top_p = 0.95
model_args.train_batch_size = 4 #8 # reducing the batch size from 8 to 4
model_args.use_multiprocessing = False
#model_args.wandb_project = "Paraphrasing with BART"

#model_args.use_cuda = False


model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,
)

model.train_model(train_df, eval_data=eval_df)

to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(eval_df["prefix"].tolist(), eval_df["input_text"].tolist())
]
truth = eval_df["target_text"].tolist()

preds = model.predict(to_predict)

# Saving the predictions if needed
os.makedirs("predictions", exist_ok=True)

with open(f"predictions/predictions_{datetime.now()}.txt", "w") as f:
    for i, text in enumerate(eval_df["input_text"].tolist()):
        f.write(str(text) + "\n\n")

        f.write("Truth:\n")
        f.write(truth[i] + "\n\n")

        f.write("Prediction:\n")
        for pred in preds[i]:
            f.write(str(pred) + "\n")
        f.write(
            "________________________________________________________________________________\n"
        )

Downloading:   0%|          | 0.00/971M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/3539 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 0.5134085009786937}
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/best_model
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/
INFO:simpletransformers.seq2seq.seq2seq_model: Training of facebook/bart-large model complete. Saved to outputs/.


Generating outputs:   0%|          | 0/443 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2343: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


In [8]:
import logging

from simpletransformers.seq2seq import Seq2SeqModel


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

model = Seq2SeqModel(
    encoder_decoder_type="bart", encoder_decoder_name="outputs"
)


while True:
    original = input("Enter text to paraphrase: ")
    to_predict = [original]

    preds = model.predict(to_predict)

    print("---------------------------------------------------------")
    print(original)

    print()
    print("Predictions >>>")
    for pred in preds[0]:
        print(pred)

    print("---------------------------------------------------------")
    print()

Enter text to paraphrase: A recording of folk songs done for the Columbia society in 1942 was largely arranged by Pjetër Dungu.


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
A recording of folk songs done for the Columbia society in 1942 was largely arranged by Pjetër Dungu.

Predictions >>>
A recording of folk songs for the Columbia Society in 1942 was largely arranged by Pjetër Dungu.
A recording of folk songs for the Columbia Society in 1942 was largely arranged by Pjetër Dungu.
A recording of folk songs for the Columbia Society in 1942 was largely arranged by Pjetër Dungu.
---------------------------------------------------------

Enter text to paraphrase: In mathematical astronomy, his fame is due to the introduction of the astronomical globe, and his early contributions to understanding the movement of the planets.


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
In mathematical astronomy, his fame is due to the introduction of the astronomical globe, and his early contributions to understanding the movement of the planets.

Predictions >>>
In mathematical astronomy, his fame is due to the introduction of the astronomical globe and his early contributions to understanding the movement of the planets.
In mathematical astronomy, his fame is due to the introduction of the astronomical globe and his early contributions to understanding the movement of the planets.
In mathematical astronomy, his fame is due to the introduction of the astronomical globe and his early contributions to understanding the movement of the planets.
---------------------------------------------------------

Enter text to paraphrase: Why are people obsessed with Cara Delevingne?


...

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------


Predictions >>>
The
The following
The
---------------------------------------------------------

Enter text to paraphrase: False


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
False

Predictions >>>
False
False
False
---------------------------------------------------------



KeyboardInterrupt: ignored

Orignal:
A recording of folk songs done for the Columbia society in 1942 was largely arranged by Pjetër Dungu.
Truth:
A recording of folk songs made for the Columbia society in 1942 was largely arranged by Pjetër Dungu.
Prediction:
A recording of folk songs made in 1942 for the Columbia Society was largely arranged by Pjetr Dungu.
A recording of folk songs for the Columbia society in 1942 was largely arranged by Pjetr Dungu.
A recording of folk songs done for the Columbia Society in 1942 was largely arranged by Pjetr Dungu.

Original:
In mathematical astronomy, his fame is due to the introduction of the astronomical globe, and his early contributions to understanding the movement of the planets.
Truth:
His fame is due in mathematical astronomy to the introduction of the astronomical globe and to his early contributions to the understanding of the movement of the planets.
Prediction:
His fame in mathematical astronomy is due to the introduction of the astronomical globe and his early contributions to understanding the movement of the planets.
In mathematical astronomy, his fame is due to the introduction of the astronomical globe and his early contributions to understanding the motion of the planets.
In mathematical astronomy his fame is due to the introduction of the astronomical globe and his early contributions to understanding the movement of the planets.



Original:
Why are people obsessed with Cara Delevingne?
Truth:
Why are people so obsessed with Cara Delevingne?
Prediction:
Why do people fall in love with Cara Delevingne?
Why is everyone obsessed with Cara Delevingne?
Why do people like Cara Delevingne?

Original:
Earl St Vincent was a British ship that was captured in 1803 and became a French trade man.
Truth:
Earl St Vincent was a British ship that was captured and became a French merchantman in 1803.
Prediction:
Earl St Vincent was a British ship captured in 1803 and became a French trader.
Earl St Vincent was a British ship captured in 1803 and became a French trade man.
Earl St Vincent was a British ship that was captured in 1803 and became a French trade man.

Original:
Worcester is a town and county city of Worcestershire in England.
Truth:
Worcester is a city and county town of Worcestershire in England.
Prediction:
Worcester is a town and county of Worcestershire in England.
Worcester is a town and county town in Worcestershire in England.
Worcester is a town and county town of Worcestershire in England.

Original:
The goal of any Deep Learning model is to take in an input and generate the correct output.
Predictions >>>
The goal of any deep learning model is to take an input and generate the correct output.
The goal of a deep learning model is to take an input and generate the correct output.
Any Deep Learning model the goal of which is to take in an input and generate the correct output.

In [ ]:
https://examples.yourdictionary.com/paraphrase-examples-rewording-sentences.html